## Pre-trained model demo

- Load the model

In [1]:
from libs.word2vec import Word2vec
from libs.data_loader import loader
from libs.seq2seq import seq2seq_chatbot
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import jieba

np.set_printoptions(precision=2)
model_name = 'test2'
dloader = loader(mode='pre_trained', model_name=model_name)

sess = tf.Session()
s2s = seq2seq_chatbot(load_model=True, model_name=model_name)
s2s.load_model(sess=sess, name='test')
word_vec, word_target, word_outputs, Attention_weights, loss_all, update_step = s2s.load_model_params_meta()

# functions for predicting next sentences
sent_len = len(word_vec)
enc_len = len(word_vec)
dec_len = len(word_target)
def next_sent(inp):
    inp = list(jieba.cut(inp))
    for j in range(len(inp)):
        if inp[j] in dloader.word2vec_lookup:
            pass
        else:
            inp[j] = '<unk>'
    if len(inp) > sent_len:
        inp = inp[0:sent_len]
    ends = ['<end>' for k in range(sent_len-len(inp))]
    inp += ends
    print(inp)
    batch_x = np.array([[dloader.word2vec_lookup[inp[i]] for i in range(len(inp))]])
    batch_y = np.array([[0 for i in range(dec_len)]])
    resp = list()
    for k in range(dec_len):
        feed_dict = {word_vec[t]: batch_x[:, t] for t in range(enc_len)}
        feed_dict.update({word_target[t]: batch_y[:, t] for t in range(dec_len)})
        
        resp.append(sess.run([word_outputs], feed_dict=feed_dict)[0][k])
        batch_y[0][k] = np.argmax(resp[k])
    atten_w = np.array(sess.run(Attention_weights, feed_dict=feed_dict)).reshape([-1])
    atten_avg = list()
    for i in range(enc_len):
        atten_avg.append(np.mean([atten_w[i+j*enc_len] for j in range(dec_len)]))
    print(np.array(atten_avg))
    sent = [dloader.oneHotPos2word[np.argmax(w)] for w in resp]
    return sent
def next_n_sent(inp, n):
    sents = list()
    for i in range(n):
        resp = next_sent(inp)
        sents.append(resp)
        inp = ''.join(sents[i]).replace('<end>', '')
    return sents

Data loader initializing
mode: pre_trained
Pre-trained dicts loading...
done
Restoring model...
done


- Testing

In [2]:
inp = '什麼時候要再過來玩'
''.join(next_sent(inp)).replace('<end>', '')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.654 seconds.
Prefix dict has been built succesfully.


['什麼', '時候', '要', '再', '過來', '玩', '<end>', '<end>', '<end>', '<end>']
[ 0.26  0.17  0.1   0.06  0.11  0.28  0.01  0.    0.    0.  ]


'我知道'

In [3]:
inp = '什麼時候要再過來玩'
''.join(next_sent(inp)).replace('<end>', '')

['什麼', '時候', '要', '再', '過來', '玩', '<end>', '<end>', '<end>', '<end>']
[ 0.25  0.16  0.1   0.06  0.13  0.28  0.02  0.    0.    0.  ]


'我知道'

In [4]:
inp = '好久不見'
''.join(next_sent(inp)).replace('<end>', '')

['好久', '不見', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.46  0.34  0.2   0.01  0.    0.    0.    0.    0.    0.  ]


'我也不知道'

In [5]:
inp = '你怎麼了'
''.join(next_sent(inp)).replace('<end>', '')

['你', '怎麼', '了', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.4   0.38  0.12  0.1   0.    0.    0.    0.    0.    0.  ]


'怎麼了'

In [6]:
inp = '為什麼'
''.join(next_sent(inp)).replace('<end>', '')

['為', '什麼', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.48  0.34  0.17  0.    0.    0.    0.    0.    0.    0.  ]


'我不是要你去相親'

In [7]:
inp = '你的工作如何'
''.join(next_sent(inp)).replace('<end>', '')

['你', '的', '工作', '如何', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.34  0.21  0.2   0.17  0.07  0.    0.    0.    0.    0.  ]


'我都會幫忙'

In [8]:
inp = '為什麼'
''.join(next_sent(inp)).replace('<end>', '')

['為', '什麼', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.53  0.32  0.14  0.    0.    0.    0.    0.    0.    0.  ]


'我是你的女朋友'

In [9]:
inp = '可是我懷疑'
''.join(next_sent(inp)).replace('<end>', '')

['可是', '我', '懷疑', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.28  0.2   0.35  0.17  0.    0.    0.    0.    0.    0.  ]


'你是一個很有自信的人'

In [10]:
inp = '覺得什麼'
''.join(next_sent(inp)).replace('<end>', '')

['覺得', '什麼', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.46  0.39  0.14  0.01  0.    0.    0.    0.    0.    0.  ]


'我看你似乎很高興'

In [11]:
inp = '真的嗎'
''.join(next_sent(inp)).replace('<end>', '')

['真的', '嗎', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.71  0.25  0.03  0.    0.    0.    0.    0.    0.    0.  ]


'當然是真的'

In [12]:
inp = '哈哈我真的好高興'
''.join(next_sent(inp)).replace('<end>', '')

['<unk>', '我', '真的', '好', '高興', '<end>', '<end>', '<end>', '<end>', '<end>']
[ 0.29  0.18  0.08  0.08  0.26  0.11  0.    0.    0.    0.  ]


'你怎麼可以這麼說呢'